<a href="https://colab.research.google.com/github/minghsu0107/ML/blob/master/machine-learning/keras_with_sklearn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Keras with Cross Validation

In [0]:
import tensorflow
import keras
 
from keras.models import Sequential
from keras.layers import Dense
import numpy as np
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from keras.wrappers.scikit_learn import KerasClassifier
 
#構建模型
def create_model():
    #構建模型
    model = Sequential()
    model.add(Dense(input_dim=8,units=12,activation='relu'))
    model.add(Dense(units=8,activation='relu'))
    model.add(Dense(units=1,activation='sigmoid'))
    
    #編譯模型
    model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
    return model
 
seed = 7
#設定隨機數種子
np.random.seed(seed)

In [27]:
url = "https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv"
names = ['preg', 'plas', 'pres', 'skin', 'test', 'mass', 'pedi', 'age', 'class']
dataframe = pd.read_csv(url, names=names)
array = dataframe.values
X = array[:,0:8]
Y = array[:,8]
dataframe.head(3)

,preg,plas,pres,skin,test,mass,pedi,age,class
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1


In [30]:
#建立模型
model = KerasClassifier(build_fn=create_model, epochs=150, batch_size=10, verbose=0)
 
#10折交叉驗證
kfold = StratifiedKFold(n_splits=2, shuffle=True, random_state=seed)
results = cross_val_score(model, X, Y, cv=kfold)
print(results.mean())

0.7135416691501935


## Keras with GridSearch

In [0]:
import tensorflow
import keras
 
from keras.models import Sequential
from keras.layers import Dense
import numpy as np
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier
 
#構建模型
def create_model(optimizer='adam',init='glorot_uniform'):
    #構建模型
    model = Sequential()
    model.add(Dense(input_dim=8,units=12,kernel_initializer=init,activation='relu'))
    model.add(Dense(units=8,kernel_initializer=init,activation='relu'))
    model.add(Dense(units=1,kernel_initializer=init,activation='sigmoid'))
    
    #編譯模型
    model.compile(loss='binary_crossentropy',optimizer=optimizer,metrics=['accuracy'])
    
    return model

In [0]:
model = KerasClassifier(build_fn=create_model,verbose=0)

In [0]:
#構建需要調參的引數
param_grid = {}
param_grid['optimizer'] = ['rmsprop','adam']
param_grid['init'] = ['glorot_uniform','normal','uniform']
param_grid['epochs'] = [1,2,4]
param_grid['batch_size'] = [5,10]

In [40]:
grid = GridSearchCV(estimator=model,param_grid=param_grid)
results = grid.fit(X,Y)

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


In [42]:
print('Best:%f using %s' %(results.best_score_,results.best_params_))
 
means = results.cv_results_['mean_test_score']
stds = results.cv_results_['std_test_score']
params = results.cv_results_['params']

for mean,std,param in zip(means,stds,params):
    print('%f (%f) with: %r' %(mean,std,param))

Best:0.671875 using {'batch_size': 5, 'epochs': 4, 'init': 'normal', 'optimizer': 'adam'}
0.427083 (0.134575) with: {'batch_size': 5, 'epochs': 1, 'init': 'glorot_uniform', 'optimizer': 'rmsprop'}
0.614583 (0.075767) with: {'batch_size': 5, 'epochs': 1, 'init': 'glorot_uniform', 'optimizer': 'adam'}
0.651042 (0.024774) with: {'batch_size': 5, 'epochs': 1, 'init': 'normal', 'optimizer': 'rmsprop'}
0.649740 (0.026557) with: {'batch_size': 5, 'epochs': 1, 'init': 'normal', 'optimizer': 'adam'}
0.651042 (0.024774) with: {'batch_size': 5, 'epochs': 1, 'init': 'uniform', 'optimizer': 'rmsprop'}
0.651042 (0.024774) with: {'batch_size': 5, 'epochs': 1, 'init': 'uniform', 'optimizer': 'adam'}
0.549479 (0.031948) with: {'batch_size': 5, 'epochs': 2, 'init': 'glorot_uniform', 'optimizer': 'rmsprop'}
0.572917 (0.086429) with: {'batch_size': 5, 'epochs': 2, 'init': 'glorot_uniform', 'optimizer': 'adam'}
0.651042 (0.024774) with: {'batch_size': 5, 'epochs': 2, 'init': 'normal', 'optimizer': 'rmsprop

## Keras with Pipeline

In [47]:
from sklearn.pipeline import Pipeline
from keras.layers import Dropout
from keras.wrappers.scikit_learn import KerasRegressor
# create a function that returns a model, taking as parameters things you
# want to verify using cross-valdiation and model selection
def create_model(optimizer='adagrad',
                 kernel_initializer='glorot_uniform', 
                 dropout=0.2):
    model = Sequential()
    model.add(Dense(64,activation='relu',kernel_initializer=kernel_initializer))
    model.add(Dropout(dropout))
    model.add(Dense(1,activation='sigmoid',kernel_initializer=kernel_initializer))

    model.compile(loss='binary_crossentropy',optimizer=optimizer, metrics=['accuracy'])

    return model

# wrap the model using the function you created
clf = KerasRegressor(build_fn=create_model,verbose=0)

# just create the pipeline
pipeline = Pipeline([
    ('clf',clf)
])

pipeline.fit(X, Y)

W0811 01:32:01.753675 140342718801792 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3445: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Pipeline(memory=None,
         steps=[('clf',
                 <keras.wrappers.scikit_learn.KerasRegressor object at 0x7fa33ec51b00>)],
         verbose=False)